# Common Meeting Info Utils


In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import json
from utils import load_certificates, get_token

In [ ]:
file_path = 'D:\\workspace\\meetinginfo-by-meetinguuid-diff.xlsx'
meeting_df = pd.read_excel(file_path)[['meetinguuid', 'siteuuid', 'eventid', 'extendinfo', 'livestreamaddr', 'meetingkey', 'vbricksiteurl', 'webexsiteurl']]
meeting_df['livestreamaddr'] = meeting_df['livestreamaddr'].fillna('')
meeting_df.head()

In [ ]:

def get_attendee_details(siteurl, eventid, token):
    """
    get attendee detail list from vbrick.
    """
    attendee_url = f'{siteurl}/api/v2/scheduled-events/{eventid}/post-event-report'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'bearer {token}'
    }
    
    records = []
    resp = requests.get(attendee_url, headers=headers)
    scrollid = resp.json()['scrollId']
    attendee_count = resp.json()['attendeeCount']
    extract_sessions(eventid, resp.json()['sessions'], records)

    while (attendee_count > 0 and scrollid is not None):
        resp = requests.get(attendee_url, headers=headers, params={'scrollId': scrollid})
        scrollid = resp.json()['scrollId']
        attendee_count = resp.json()['attendeeCount']
        extract_sessions(eventid, resp.json()['sessions'], records)
    return records


def extract_sessions(eventid, sessions, records):
    for session in sessions:
        record = {}
        record['eventid'] = eventid
        record['user_type'] = session['userType']
        record['name'] = session['name']
        record['email'] = session['email']
        record['ip_address'] = session['ipAddress']
        record['browser'] = session['browser']
        record['device_type'] = session['deviceType']
        record['zone'] = session['zone']
        record['entered_date'] = session['enteredDate']
        record['exited_date'] = session['exitedDate']
        record['session_time'] = session['sessionTime']
        record['viewing_start_time'] = session['viewingStartTime']
        record['viewing_time'] = session['viewingTime']
        record['public_cdn_time'] = session['publicCDNTime']
        record['ecdn_time'] = session['eCDNTime']
        record['attendee_type'] = session['attendeeType']
        records.append(record)


def get_event_info(siteurl, eventid, token):
    """
    get event info from vbrick.
    """
    event_url = f'{siteurl}/api/v2/scheduled-events/{eventid}'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'bearer {token}'
    }
    resp = requests.get(event_url, headers=headers)
    if resp.status_code == 200:
        return resp.json()
    else:
        return None



def extract_properties(meeting_info, event_info):
    record = {}
    # meeting info
    record['meeting_uuid'] = meeting_info.meetinguuid
    extend_json = json.loads(meeting_info.extendinfo)
    record['start_time'] = extend_json['startTime']
    record['end_time'] = extend_json['endTime']
    record['meeting_key'] = meeting_info.meetingkey
    record['site_uuid'] = meeting_info.siteuuid
    record['eventid']= meeting_info.eventid
    record['vbrick_site_url'] = meeting_info.vbricksiteurl
    record['webex_site_url'] = meeting_info.webexsiteurl

    # event info
    record['title'] = event_info['title']
    record['polls_enabled'] = event_info['pollsEnabled']
    record['chat_enabled'] = event_info['chatEnabled']
    record['qa_enabled'] = event_info['questionAndAnswerEnabled']
    record['schedule_moderator_count'] = len(event_info['moderatorIds'])
    record['autoplay'] = event_info['autoplay']
    record['custom_branding_enabled'] = event_info['enableCustomBranding']
    return record       

In [ ]:
%%time
"""
loop dataframe to fetch meeting details
"""


print(f'loop begin: total rows is {meeting_df.shape[0]}')

certificates = load_certificates('D:\\workspace\\certificates.json')
meeting_records = []
attendee_records = []


for row in meeting_df.itertuples():
    record = {}

    vbricksiteurl = row.vbricksiteurl
    region = re.search('https://(.+?).livestream', vbricksiteurl).group(1)
    eventid = row.eventid
    livestreamaddr = row.livestreamaddr
    token = get_token(vbricksiteurl, region, certificates)

    try:
        event_resp = get_event_info(vbricksiteurl, eventid, token)
        if event_resp == None:
            print(f'something went wrong, event {eventid}, status {event_resp.status}')
        else:
            if livestreamaddr == '':
                attendee_list = []
            else:
                attendee_list = get_attendee_details(vbricksiteurl, eventid, token)
            
            record = extract_properties(row, event_resp)
            meeting_records.append(record)
            attendee_records.extend(attendee_list)
    except Exception as e:
        print(f'error thrown event {eventid}')
        print(e)

    
meet_df = pd.DataFrame(meeting_records)
meet_df.to_excel('meet-list.xlsx', index=False)
attendee_df = pd.DataFrame(attendee_records)
attendee_df.to_excel('att-list.xlsx', index=False)